In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import transformers
from transformers import BartTokenizerFast

from model import BartSummaryModelV2
from inference import get_top_k_sentences, extract_sentences
from dataset import SummaryDataset
from utils import collate_fn
from truncate import (
    batch_truncate, 
    batch_truncate_with_eq, 
    batch_truncate_with_len, 
    gather_lengths,
    concat_sentences,
)

In [2]:
MODEL_NAME = "gogamza/kobart-summarization"
MODEL_PATH = "../saved"

tokenizer = BartTokenizerFast.from_pretrained(MODEL_NAME)
model = BartSummaryModelV2.from_pretrained(MODEL_PATH)

model = model.cuda()

In [3]:
dataset = SummaryDataset("/opt/ml/dataset/Training/train.parquet", tokenizer, is_train=True, truncate=False)
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=lambda x: collate_fn(x, tokenizer.pad_token_id))

In [4]:
batch = next(iter(dataloader))

In [5]:
input_ids = batch["input_ids"].cuda()
attention_mask = batch["attention_mask"].cuda()
answers = batch["answers"].cuda()

In [34]:
def get_eos_positions(x: torch.Tensor, tokenizer: BartTokenizerFast):
    eos_positions = []
    for i in range(x.size(0)):
        ids = torch.eq(x[i], tokenizer.eos_token_id).nonzero().squeeze(1)
        eos_positions.append(ids)
    return torch.nn.utils.rnn.pad_sequence(eos_positions, batch_first=True, padding_value=-1)

In [47]:
MAX_LEN = 256
model.eval()

summaries = []

for idx, batch in enumerate(dataloader):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    answers = batch["answers"]

    print(f"Starting step {idx}")
    while input_ids is not None:

        _input_ids, input_ids, mapping = batch_truncate_with_eq(
            input_ids, 
            MAX_LEN, 
            sep=tokenizer.eos_token_id, 
            padding_value=tokenizer.pad_token_id, 
            eos_value=tokenizer.eos_token_id, 
            return_mapping=True
        )

        lengths = gather_lengths(_input_ids, tokenizer.pad_token_id)

        _attention_mask = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor([1.0] * l) for l in lengths], 
            batch_first=True, 
            padding_value=0.0
        ).to(_input_ids.device)

        _input_ids_c = _input_ids.cuda()
        _attention_mask_c = _attention_mask.cuda()
        print(f"_input_ids.shape: {_input_ids_c.shape}, _attention_mask.shape: {_attention_mask_c.shape}")
        print(f"{tokenizer.decode(_input_ids[0].tolist())}")

        print(f"_input_ids_c: {_input_ids_c.device}, _input_ids: {_input_ids.device}")

        ext_out = model.classify(
            input_ids=_input_ids_c, 
            attention_mask=_attention_mask_c,
        )
        print([f"{k}: {v.shape}" for k, v in ext_out.items() if isinstance(v, torch.Tensor)])

        _eos_positions = get_eos_positions(_input_ids, tokenizer)
        print(_eos_positions)
        
        top_ext_ids = get_top_k_sentences(
            logits=ext_out.logits.clone().detach().cpu(),
            eos_positions=_eos_positions,
            k=3,
        )
        print(f"top_ext_ids: {top_ext_ids[0]}")
        _ext_batch = extract_sentences(_input_ids, _eos_positions, top_ext_ids, tokenizer)
        print(f"_ext_batch.shape: {_ext_batch['input_ids'].shape}")
        
        if input_ids is not None:
            input_ids = concat_sentences(_ext_batch["input_ids"], input_ids, tokenizer.pad_token_id)
            continue
        
        gen_batch = _ext_batch

    summary_ids = model.generate(
        input_ids=gen_batch["input_ids"].cuda(), 
        attention_mask=gen_batch["attention_mask"].cuda(), 
        num_beams=8, 
        max_length=128, 
        min_length=4,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
    )
    summary_sent = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.extend(summary_sent)

    if idx > 5:
        break


Starting step 0
_input_ids.shape: torch.Size([4, 242]), _attention_mask.shape: torch.Size([4, 242])
<s>전남드래곤즈 해맞이 다짐...선수 영입 활발</s>이성훈 sinawi@hanmail.net</s>전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다.</s>임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를 달성하기 위한 결연한 의지를 다졌다.</s>이번 해맞이 행사는 2018년을 시작하면서 떠오르는 해를 보며 전남드래곤즈 구성원 모두가 한마음 한 뜻으로 구단 목표 달성을 위해 정진하자는 의미에서 실시한 것이다.</s>신승재 사장은“유상철 감독을 비롯한 코칭스텝, 선수단 구성이 마무리 된 만큼 구성원 모두가 하나되어 K리그 클래식 5위 이내 진입, FA컵 우승 등 ACL 진출권 획득을 목표로 최선을 다하자”고 선수들에게 신년 인사말을 전했다.</s>유상철 감독은“구봉산의 정기를 받아 2018년을 전남드래곤즈의 해로 만들겠다”며 각오를 다졌다.</s>한편 전남은 선수들도 추가 영입했다.</s>우선 프렌차이즈 스타 김영욱과 2020년까지 연장계약을 했다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
_input_ids_c: cuda:0, _input_ids: cpu
['logits: torch.Size([4, 9])', 'encoder_last_hidden_state: torch.Size([4, 242, 768])']
tensor([[ 13,  30,  55,  83, 118, 176, 201, 211, 226],
        [ 14,  31,  53,  87, 122, 153, 199, 241,  -1],
        [ 18,  35,  63,  94, 130, 149, 181, 217, 240],
        

In [48]:
summaries

['전남드래곤즈는 지난 4일 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를 달성하기 위한 결연한 의지를 다졌으며, 이번 해맞이 행사는 2018년을 시작하면서 떠오르는 해를 보며 전남드래곤즈 구성원 모두가 한마음 한 뜻으로 구단 목표 달성을 위해 정진하자는 의미에서 실시한 것이다.',
 '전라남도가 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도인 쌀 생산조정제를 적극 추진키로 했다.',
 "여수시는 '낮에는 색채, 밤에는 빛'을 주제로 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의 도시를 완성했다.",
 '광양시는 농산물 FTA 확대와 경기침체 등 국내외 농업여건의 어려워짐에 따라 농업인들의 경쟁력을 높이고, 소득안정을 위해 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050명을 대상으로 새해 농업인 실용교육을 실시한다.',
 '태소 시내버스를 애용하는 A 씨를‘열 받게 한 것’은 1월 1일자로 변경된 시내버스 일부 노선이 사전에 충분한 공지 없이 감차가 됐기 때문인데 11-1번은 태인동, 금호동, 중마동을 거쳐 광양읍으로 가는 유일한 노선인데 운행횟수를 13회나 줄였기 때문에 태인동 주민들이 광양5일장에 나가기가 힘들고 광양, 순천으로 통학하는 학생들의 통학이 어려워 큰 불편을 겪게 됐다.',
 '전라남도는 신규시장인 타이완 크루즈 유치를 위해 해양수산부와 공동으로 지난해 상반기부터 타이완에서 크루즈포트세일, 크루즈협회·여행사 방문세일즈를 진행했으며 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가 타이완 관광객 4000여명을 싣고 대만 지룽(基龍)항을 출발해 여수를 방문한다고 밝혔다.',
 '광양교육지원청 Wee센터는 초등학교 고학년(4~6학년)·중학생 및 학부모를 대상으로 성격검사, 진로검사, 양육태도 검사를 실시한 후 1:1해석 상담을 통해 부모-자녀간 이해 및 자기이해 증진을 위한 프로그램을 진행하는 겨울방학 카운

In [33]:
ids = torch.eq(batch["input_ids"], tokenizer.eos_token_id).nonzero()
ids

tensor([[  0,  13],
        [  0,  30],
        [  0,  55],
        [  0,  83],
        [  0, 118],
        [  0, 176],
        [  0, 201],
        [  0, 211],
        [  0, 226],
        [  0, 263],
        [  0, 285],
        [  0, 307],
        [  0, 332],
        [  0, 354],
        [  0, 392],
        [  0, 418],
        [  0, 443],
        [  0, 462],
        [  1,  14],
        [  1,  31],
        [  1,  53],
        [  1,  87],
        [  1, 122],
        [  1, 153],
        [  1, 199],
        [  1, 241],
        [  1, 281],
        [  1, 300],
        [  1, 329],
        [  1, 369],
        [  1, 425],
        [  2,  18],
        [  2,  35],
        [  2,  63],
        [  2,  94],
        [  2, 130],
        [  2, 149],
        [  2, 181],
        [  2, 217],
        [  2, 240],
        [  2, 259],
        [  2, 275],
        [  2, 298],
        [  2, 356],
        [  3,  19],
        [  3,  36],
        [  3,  67],
        [  3,  98],
        [  3, 131],
        [  3, 169],
